# Notebook 10: Agent Orchestration with LangGraph

Purpose:
- Convert NEEL’s backend logic into an explicit agent execution graph
- Enforce Supervisor-first reasoning
- Inject memory and analytics safely
- Prevent uncontrolled LLM execution

This notebook is fully self-contained.

In [1]:
# !pip install langchain langgraph

In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
import os
from dotenv import load_dotenv
from google import genai

In [3]:
load_dotenv()

GEMINI_API_KEY = os.getenv("Google_Gemini_Api_Key")
if not GEMINI_API_KEY:
    raise ValueError("Gemini API key not found in .env file")

client = genai.Client(api_key=GEMINI_API_KEY)

In [4]:
class AgentState(TypedDict):
    profile: dict
    memory: dict
    analytics: dict
    supervisor: dict
    llm_response: str

In [5]:
user_profile = {
    "education": "MCA",
    "goal": "Become an ML Engineer",
    "priority": "Learning + Health"
}

In [6]:
analytics_summary = {
    "avg_work_minutes": 421,
    "avg_leisure_minutes": 255,
    "avg_sleep_minutes": 426,
    "avg_exercise_minutes": 64,
    "mean_productivity": 82
}

In [7]:
supervisor_decision = {
    "status": "ALLOW",     
    "confidence": "LOW",   
    "warnings": [
        "Productivity instability detected",
        "Conflict: high output but reported fatigue"
    ]
}

In [8]:
memory_payload = {
    "short_term": {
        "days_observed": 5,
        "avg_work_minutes": 490,
        "avg_sleep_minutes": 356,
        "avg_exercise_minutes": 18,
        "mean_productivity": 81,
        "productivity_trend": "stable"
    },
    "reflective": "Sleep duration has been consistently low; Physical activity levels are minimal",
    "profile": user_profile
}

In [9]:
def build_reasoning_prompt(profile, analytics, supervisor):
    warnings_text = (
        "\n".join(f"- {w}" for w in supervisor["warnings"])
        if supervisor["warnings"]
        else "None"
    )

    return f"""
You are NEEL, a cautious and responsible AI assistant.

USER CONTEXT:
- Education: {profile['education']}
- Goal: {profile['goal']}
- Priority: {profile['priority']}

ANALYTICS SUMMARY:
- Average Work Time: {analytics['avg_work_minutes']} minutes/day
- Average Leisure Time: {analytics['avg_leisure_minutes']} minutes/day
- Average Sleep Time: {analytics['avg_sleep_minutes']} minutes/day
- Average Exercise Time: {analytics['avg_exercise_minutes']} minutes/day
- Mean Productivity Score: {analytics['mean_productivity']}

SUPERVISOR STATUS:
- Confidence Level: {supervisor['confidence']}
- Warnings:
{warnings_text}

INSTRUCTIONS:
- Do NOT give medical or absolute advice
- Be cautious if confidence is LOW
- Explicitly acknowledge uncertainty
- Do NOT mention ML models or scores
- Keep tone professional and calm

STYLE GUIDELINES:
- At most one subtle, professional emoji may be used if appropriate
- Avoid emojis in sensitive or uncertain situations
- Emojis are optional

RESPONSE FORMAT:
OBSERVATION:
REASONING:
SUGGESTION:
CONFIDENCE NOTE:
"""

In [10]:
def analytics_node(state: AgentState) -> AgentState:
    return state

In [11]:
def supervisor_node(state: AgentState) -> AgentState:
    if state["supervisor"]["status"] != "ALLOW":
        state["llm_response"] = "Reasoning blocked by Supervisor."
    return state

In [12]:
def llm_reasoning_node(state: AgentState) -> AgentState:
    prompt = build_reasoning_prompt(
        state["profile"],
        state["analytics"],
        state["supervisor"]
    )

    response = client.models.generate_content(
        model="models/gemini-2.5-flash",
        contents=prompt
    )

    state["llm_response"] = response.text
    return state

In [13]:
def route_after_supervisor(state: AgentState):
    if state["supervisor"]["status"] == "ALLOW":
        return "llm_reasoning"
    return END

In [14]:
graph = StateGraph(AgentState)

graph.add_node("analytics", analytics_node)
graph.add_node("supervisor", supervisor_node)
graph.add_node("llm_reasoning", llm_reasoning_node)

graph.set_entry_point("analytics")

graph.add_edge("analytics", "supervisor")
graph.add_conditional_edges(
    "supervisor",
    route_after_supervisor,
    {
        "llm_reasoning": "llm_reasoning",
        END: END
    }
)

graph.add_edge("llm_reasoning", END)

neel_agent = graph.compile()

In [15]:
initial_state = {
    "profile": user_profile,
    "memory": memory_payload,
    "analytics": analytics_summary,
    "supervisor": supervisor_decision,
    "llm_response": ""
}

final_state = neel_agent.invoke(initial_state)
final_state["llm_response"]

"OBSERVATION:\nYour analytics indicate a substantial dedication to work (averaging 421 minutes/day) and exercise (64 minutes/day). However, the supervisor's feedback highlights a significant concern: reported fatigue despite high output, coupled with detected productivity instability. Your average sleep duration is noted at 426 minutes (approximately 7 hours).\n\nREASONING:\nYour goal to become an ML Engineer, coupled with your priority for learning and health, suggests a need for sustainable routines. While your work output is high, consistently operating at a demanding pace with a sleep duration that is on the lower end of general recommendations (7-9 hours for adults) could contribute to the reported fatigue. Fatigue, in turn, may naturally lead to fluctuations in productivity and consistency, even if overall output remains high. Maintaining optimal health is fundamental for sustained learning and progress towards your career objectives.\n\nSUGGESTION:\nConsidering your priority for

This execution demonstrates:

- Explicit agent orchestration
- Supervisor-gated reasoning
- Memory-aware context
- Deterministic execution flow

NEEL is now a true agentic backend, not a prompt-based chatbot.